 importando bibliotecas para raspagem de alugueis na [Imobiliaria Ativa](https://www.imobiliariaativa.com.br/)

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


class_imovel = 'muda_card1.ms-lg-0.col-12.col-md-12.col-lg-6.col-xl-4.mt-4.d-flex.align-self-stretch.justify-content-center'

def getPage(n):
    return f'https://www.imobiliariaativa.com.br/pesquisa-de-imoveis/?locacao_venda=L&finalidade=&dormitorio=&garagem=&vmi=&vma=&ordem=6&&pag={n}'

def percorra(driver):
    have = True
    locs = 0
    pag = 0
    while have:
        try:
            pag += 1
            driver.get(getPage(pag))
            wait = WebDriverWait(driver,30)
            imoveis = driver.find_elements(By.CLASS_NAME, class_imovel)

            print(locs := len(imoveis)+locs)
            if locs >= int(driver.find_element(By.XPATH,'/html/body/main/section/div/h1').text.split(' ')[0]):
                return
            yield imoveis
        finally:
            # driver.quit()
            pass
def calcularTipo(titulo):
    lTitle = titulo.lower()
    if 'casa' in lTitle:
        return 'Casa'
    elif 'comercia' in lTitle:
        return 'Sala Comercial'
    elif 'barracão' in lTitle:
        return 'Barracão'
    elif 'sobrado' in lTitle:
        return 'Sobrado'
    elif 'apartamento' in  lTitle:
        return 'Apartamento'

def buscarCondominio(link,driver,bustarArea,area,tipo):
    driver.execute_script(f"window.open('{link}');")
    driver.switch_to.window(driver.window_handles[1])
    condominio = 'None'
    
    if tipo == 'Apartamento':
        try:
            for i in range(2,10):
                if driver.find_element(By.XPATH,f'//*[@id="valores_imovel"]/div[{i}]/div[1]').text ==  'Condomínio':
                    condominio = driver.find_element(By.XPATH,f'//*[@id="valores_imovel"]/div[{i}]/div[2]').text
                    print(condominio)
                    break
        except:
            print('Nao encontrado')
            
 
    # area = ''
    if bustarArea:
        area = [area 
            for 
            area in [e.text for e in driver.find_elements(By.CLASS_NAME,'fw-bold')] 
            if ' m²' in area]
        if len(area)>0:
            area = area[0]
            # print(area)
        else:
            descricao = driver.find_element(By.CLASS_NAME,'descricao-imovel').text
            if ' m²' in descricao:
                palavras = descricao.split()
                for i,palavra in enumerate(palavras):
                    if palavra == 'm²':
                        area = f'{palavras[i-1].replace(',','.')} {palavras[i]}'
            else:
                area = None

    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    return ('R$ '+condominio.replace(',','.'),area)

def getAttrs(imovel,driver):
    titulo = imovel.find_element(By.CLASS_NAME,'card-titulo.corta-card-titulo').text.replace(',','.')
    tipo = calcularTipo(titulo)
    link = imovel.find_element(By.CLASS_NAME,'carousel-cell.is-selected').get_attribute('href')
    buscarArea = False
    suites = ''
    quartos = ''
    area = ''
    condominio = 'None'
    extras  = imovel.find_element(By.CLASS_NAME,'imo-dad-compl')
    for extra in extras.find_elements(By.TAG_NAME,'div'):
        if 'dorm-ico' in extra.get_attribute('class').split():
            quartos = extra.find_elements(By.TAG_NAME,'span')[1].text
        if 'suites-ico' in extra.get_attribute('class').split():
            suites = extra.find_elements(By.TAG_NAME,'span')[1].text
        if 'a-terr-ico' in extra.get_attribute('class').split():
            area = extra.find_elements(By.TAG_NAME,'span')[1].text
        elif 'a-const-ico' in extra.get_attribute('class').split():
            area = extra.find_elements(By.TAG_NAME,'span')[1].text
        elif 'a-util-ico' in extra.get_attribute('class').split():
            area = extra.find_elements(By.TAG_NAME,'span')[1].text
        elif 'a-total-ico' in extra.get_attribute('class').split():
            area = extra.find_elements(By.TAG_NAME,'span')[1].text
        else:
            buscar_area = True

    valor = imovel.find_element(By.CLASS_NAME,'card-valores').text.split(',')[0]+'.00'

    condominio,area = buscarCondominio(link,driver,buscarArea,area,tipo)
    # condominio = condominio if tipo == 'Apartamento' else 'None'
    bairro = imovel.find_element(By.CLASS_NAME,'card-bairro-cidade-texto').text
    return ','.join([str(e) for e in [titulo,tipo,valor,condominio,quartos,suites,area,bairro,link,'Imobiliaria Ativa\n']] )
    

driver = webdriver.Chrome()
driver.maximize_window()
cabecalho = 'Titulo do imóvel,Tipo,Preço,Valor Condominio,Quartos,Suítes,Área Total,Bairro,Link para o anúncio,Origem\n'
try:
    with open('out.csv','w',encoding='utf-8') as out:
        out.write(cabecalho)

        for imoveis in percorra(driver):
            # print (imoveis)
            for imovel in imoveis:
                out.write(getAttrs(imovel,driver))



finally:
    driver.quit()




27
[<selenium.webdriver.remote.webelement.WebElement (session="057224e6a3ad1fe5da0990aad05875cc", element="f.29DBD69BE729195763C8487E9D16A582.d.C69CB8D29FC2CBADD9F211FBA0995647.e.96")>, <selenium.webdriver.remote.webelement.WebElement (session="057224e6a3ad1fe5da0990aad05875cc", element="f.29DBD69BE729195763C8487E9D16A582.d.C69CB8D29FC2CBADD9F211FBA0995647.e.97")>, <selenium.webdriver.remote.webelement.WebElement (session="057224e6a3ad1fe5da0990aad05875cc", element="f.29DBD69BE729195763C8487E9D16A582.d.C69CB8D29FC2CBADD9F211FBA0995647.e.98")>, <selenium.webdriver.remote.webelement.WebElement (session="057224e6a3ad1fe5da0990aad05875cc", element="f.29DBD69BE729195763C8487E9D16A582.d.C69CB8D29FC2CBADD9F211FBA0995647.e.99")>, <selenium.webdriver.remote.webelement.WebElement (session="057224e6a3ad1fe5da0990aad05875cc", element="f.29DBD69BE729195763C8487E9D16A582.d.C69CB8D29FC2CBADD9F211FBA0995647.e.100")>, <selenium.webdriver.remote.webelement.WebElement (session="057224e6a3ad1fe5da0990aad0